# Неделя 3. Практическая работа 
## Визуализация данных 

Импортируем необходимые библиотеки

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


# не нужно вызывать plt.show() для отрисовки графиков.
%matplotlib inline


#не печатаем warnings
import warnings
warnings.filterwarnings('ignore')

### Titanic dataset

Прочитаем данные 

In [ ]:
titanic_data = pd.read_csv('data/titanic_train.csv')

In [ ]:
titanic_data.head(10)

Посмотрим сведения о признаках

In [ ]:
titanic_data.info()

Посчитаем и визуализируем пропуски в данных

In [ ]:
null_info = titanic_data.isna().sum()
null_info

In [ ]:
plt.figure(figsize=(15,3)) # размер графика, в дюймах
plt.bar(null_info.index, null_info.values)

Изменим стандартный стиль отображения графиков

In [ ]:
%config InlineBackend.figure_format = 'svg' 

Отсортируем данные о пропусках, выведем в долях, улучшим читаемость графика

In [ ]:
null_info.sort_values(inplace=True,ascending=False) #сортировка, параметр inplace - сортировка конкретного объекта pd.Series
null_info = null_info/titanic_data.shape[0]


plt.figure(figsize=(12,3)) # размер графика, в дюймах
plt.bar(null_info.index, null_info.values)
plt.title('доля пропусков по признакам')
plt.grid()

### Seaborn

выведем распределение возрастов в seaborn

In [ ]:
sns.distplot(titanic_data.Age.dropna(),bins=40) # необходимо удалить пропуски

Посмотрим на статистики возрастов в зависисмости от класса пассажира

In [ ]:
pclass_age_stats = titanic_data.groupby('Pclass').apply(lambda x: x.Age.describe())
pclass_age_stats.head()

In [ ]:
sns.barplot(pclass_age_stats.index, pclass_age_stats['mean'])

Но можно сделать значительно лучше

In [ ]:
sns.boxplot(titanic_data.Pclass, titanic_data.Age)

In [ ]:
sns.pairplot(titanic_data)

### RN Hackaton dataset

In [ ]:
data = pd.read_csv('data/contest_train_df.csv',parse_dates=['Start_date','Date'])
data.head(10)

In [ ]:
data.info()

In [ ]:
null_info = data.isna().sum()

null_info.sort_values(inplace=True,ascending=False)
null_info = null_info/data.shape[0]

null_info = null_info[:12] # выведем только первые 12 признаков

plt.figure(figsize=(14,3)) 
plt.bar(null_info.index, null_info.values)
plt.title('доля пропусков по признакам')
plt.grid()

Для каждой скважины известна история ее работы. Проверим длинну истории

In [ ]:
history_lens = data.groupby('id').apply(lambda x: x.shape[0])
sns.distplot(history_lens,bins=30, kde=False)

In [ ]:
history_lens.describe()

Посмотрим как изменяется число запустившихся скважин по годам

In [ ]:
data['start_year'] = data.Start_date.apply(lambda x: x.year)

In [ ]:
data.start_year.sort_index().value_counts()

In [ ]:
start_count_by_year = data.drop_duplicates('id').start_year.sort_index().value_counts()

In [ ]:
plt.plot(start_count_by_year.index, start_count_by_year.values,'o')
plt.ylabel('число стартовавших скважин')
plt.xlabel('год')
plt.grid()

Проверим корреляцию между числовыми признаками

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(data.corr())

#### Q_OIS на дату ВНР (VNR==1) - наша целевая переменная(то, что мы хотим прогнозировать), проверим ее значения 

In [ ]:
plt.suptitle('Q_OIS для нескольких скважин')


ids = [7,10,15]
plt.figure(figsize=(10,10))

for i,id in enumerate(ids):
    print(i,id)
    plt.subplot(3,1,i+1)
    plt.title('id = '+str(id))

    one_well_data = data[data.id==id]
    one_well_data.sort_values('Date',inplace=True)
    plt.plot(one_well_data.Date, one_well_data.Q_OIS)
    plt.grid()



In [ ]:
data[data.VNR==1].Q_OIS.describe()

In [ ]:
sns.distplot(data[data.VNR==1].Q_OIS)

Попробуем построить наивную модель

In [ ]:
def calc_mean_q_oist(chunk):
    target = chunk[chunk.VNR==1].Q_OIS.values[0]
    mean_qois = chunk[chunk.VNR!=1].Q_OIS.mean()
    
    return {'mean_q_OIS': mean_qois,'target':target}

In [ ]:
dataset = data.groupby('id').apply(calc_mean_q_oist)
dataset = pd.DataFrame.from_records(dataset.values)

In [ ]:
dataset.head()

In [ ]:
plt.scatter(dataset.mean_q_OIS, dataset.target)
plt.grid()
plt.xlabel('mean_q')
plt.ylabel('target')

In [ ]:
dataset.corr()

### Понижение размерности с помощью метода главных компонент (PCA)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
features_to_drop = ['state_age', 'Date', 'Start_date', 'Well_ID', 'id', 'VNR','Formation_lbl','State','IDN_type']

pca = PCA()
transformed_data = pca.fit_transform(data.drop(features_to_drop, axis=1).dropna())

transformed_data.shape

Посмотрим на долю объясненной дисперсии

In [ ]:
pca.explained_variance_ratio_

Визуализируем!

In [ ]:
sns.jointplot(transformed_data[:,0], transformed_data[:,1], kind='kde')